In [ ]:
#default_exp model.rt

In [ ]:
#hide
#automatically reload modules so that when changing some modules you do not need to explicitly reload or reset the ipython kernel
%reload_ext autoreload
%autoreload 2

# Retention Time Prediction Models

This notebook implements deep learning models to predict the retention time of a given (modified) peptide. The components of the models are defined in the `building_block.ipynb` notebook. It is based on the interface defined in `model_interface.ipynb`, which defines standard interactions, such as loading data and training the model.

## Imports

In [ ]:
#export
import torch
import pandas as pd
import numpy as np

from peptdeep.model.featurize import (
    get_batch_aa_indices, 
    get_batch_mod_feature
)

from peptdeep.settings import model_const

import peptdeep.model.model_interface as model_interface
import peptdeep.model.building_block as building_block

mod_feature_size = len(model_const['mod_elements'])

## Constants
irt peptides as reported by [Escher et al.]('https://doi.org/10.1002/pmic.201100463')

In [ ]:

#export


IRT_PEPTIDE_DF = pd.DataFrame(
    [['LGGNEQVTR', 'RT-pep a', -24.92, '', ''],
    ['GAGSSEPVTGLDAK', 'RT-pep b', 0.00, '', ''],
    ['VEATFGVDESNAK', 'RT-pep c', 12.39, '', ''],
    ['YILAGVENSK', 'RT-pep d', 19.79, '', ''],
    ['TPVISGGPYEYR', 'RT-pep e', 28.71, '', ''],
    ['TPVITGAPYEYR', 'RT-pep f', 33.38, '', ''],
    ['DGLDAASYYAPVR', 'RT-pep g', 42.26, '', ''],
    ['ADVTPADFSEWSK', 'RT-pep h', 54.62, '', ''],
    ['GTFIIDPGGVIR', 'RT-pep i', 70.52, '', ''],
    ['GTFIIDPAAVIR', 'RT-pep k', 87.23, '', ''],
    ['LFLQFGAQGSPFLK', 'RT-pep l', 100.00, '', '']],
    columns=['sequence','pep_name','irt', 'mods', 'mod_sites']
)
IRT_PEPTIDE_DF['nAA'] = IRT_PEPTIDE_DF.sequence.str.len()


#legacy
irt_pep = IRT_PEPTIDE_DF


## Model Definitions

In the model definition classes, all layers of the model are defined, utilizing the "building blocks" defined in `building_blocks.ipynb`

### Transformer based model


In [ ]:
#export
class Model_RT_Bert(torch.nn.Module):
    def __init__(self,
        dropout = 0.1,
        nlayers = 4,
        hidden = 128,
        output_attentions=False,
        **kwargs,
    ):
        """
        Model based on a transformer Architecture from 
        Huggingface's BertEncoder class.
        """
        super().__init__()

        self.dropout = torch.nn.Dropout(dropout)

        self.input_nn = building_block.AATransformerEncoding(hidden)

        self._output_attentions = output_attentions
        
        self.hidden_nn = building_block.Hidden_HFace_Transformer(
            hidden, nlayers=nlayers, dropout=dropout,
            output_attentions=output_attentions
        )

        self.output_nn = torch.nn.Sequential(
            building_block.SeqAttentionSum(hidden),
            torch.nn.PReLU(),
            self.dropout,
            torch.nn.Linear(hidden, 1),
        )

    @property
    def output_attentions(self):
        return self._output_attentions

    @output_attentions.setter
    def output_attentions(self, val:bool):
        self._output_attentions = val
        self.hidden_nn.output_attentions = val

    def forward(self, 
        aa_indices, 
        mod_x,
    ):
        x = self.dropout(self.input_nn(
            aa_indices, mod_x
        ))

        hidden_x = self.hidden_nn(x)
        if self.output_attentions:
            self.attentions = hidden_x[1]
        else:
            self.attentions = None
        x = self.dropout(hidden_x[0]+x*0.2)

        return self.output_nn(x).squeeze(1)


### LSTM model 

In [ ]:
#export
class Model_RT_LSTM_CNN(torch.nn.Module):
    def __init__(self, 
        dropout=0.2,
    ):
        """
        Model based on a combined CNN/LSTM architecture
        """
        super().__init__()
        
        self.dropout = torch.nn.Dropout(dropout)
        
        hidden = 256
        self.rt_encoder = building_block.Encoder_26AA_Mod_CNN_LSTM_AttnSum(
            hidden
        )

        self.rt_decoder = building_block.Decoder_Linear(
            hidden,
            1
        )

    def forward(self, 
        aa_indices, 
        mod_x,
    ):
        x = self.rt_encoder(aa_indices, mod_x)
        x = self.dropout(x)

        return self.rt_decoder(x).squeeze(1)
#legacy
Model_RT_LSTM = Model_RT_LSTM_CNN

## Model Pipeline
The final model which is used to interact with the data is defined below, utilizing the `ModelInterface` class defined in `model_interface.ipynb` 

In [ ]:
#export
class AlphaRTModel(model_interface.ModelInterface):
    def __init__(self, 
        dropout=0.1,
        model_class:torch.nn.Module=Model_RT_LSTM_CNN, #model defined above
        device:str='gpu',
        **kwargs,
    ):
        """
        Class to predict retention times from precursor dataframes.
        """
        super().__init__(device=device)
        self.build(
            model_class,
            dropout=dropout,
            **kwargs
        )
        self.target_column_to_predict = 'rt_pred'
        self.target_column_to_train = 'rt_norm'

    def _get_features_from_batch_df(self, 
        batch_df: pd.DataFrame,
    ):
        return (
            self._get_26aa_indice_features(batch_df),
            self._get_mod_features(batch_df)
        )

    def add_irt_column_to_precursor_df(self,
        precursor_df: pd.DataFrame
    ):
        self.predict(IRT_PEPTIDE_DF)
        # simple linear regression
        rt_pred_mean = IRT_PEPTIDE_DF.rt_pred.mean()
        irt_mean = IRT_PEPTIDE_DF.irt.mean()
        x = IRT_PEPTIDE_DF.rt_pred.values - rt_pred_mean
        y = IRT_PEPTIDE_DF.irt.values - irt_mean
        slope = np.sum(x*y)/np.sum(x*x)
        intercept = irt_mean - slope*rt_pred_mean
        # end linear regression
        precursor_df['irt_pred'] = precursor_df.rt_pred*slope + intercept
        return precursor_df



## Unit Tests

### Sanity checks on simple cases

In [ ]:
#hide
def initialize_and_print_model():

    torch.manual_seed(1337)
    model = AlphaRTModel()
    model.device = torch.device('cpu')
    model.model.to(model.device)
    mod_hidden = len(model_const['mod_elements'])
    model.model(torch.LongTensor([[1,2,3,4,5,6]]), torch.tensor([[[0.0]*mod_hidden]*6]))
    print(model.get_parameter_num())
    print(model.model)
    return model



model = initialize_and_print_model()


708224
Model_RT_LSTM_CNN(
  (dropout): Dropout(p=0.1, inplace=False)
  (rt_encoder): Encoder_AA_Mod_CNN_LSTM_AttnSum(
    (mod_nn): Mod_Embedding_FixFirstK(
      (nn): Linear(in_features=103, out_features=2, bias=False)
    )
    (input_cnn): SeqCNN(
      (cnn_short): Conv1d(35, 35, kernel_size=(3,), stride=(1,), padding=(1,))
      (cnn_medium): Conv1d(35, 35, kernel_size=(5,), stride=(1,), padding=(2,))
      (cnn_long): Conv1d(35, 35, kernel_size=(7,), stride=(1,), padding=(3,))
    )
    (hidden_nn): SeqLSTM(
      (rnn): LSTM(140, 128, num_layers=2, batch_first=True, bidirectional=True)
    )
    (attn_sum): SeqAttentionSum(
      (attn): Sequential(
        (0): Linear(in_features=256, out_features=1, bias=False)
        (1): Softmax(dim=1)
      )
    )
  )
  (rt_decoder): Decoder_Linear(
    (nn): Sequential(
      (0): Linear(in_features=256, out_features=64, bias=True)
      (1): PReLU(num_parameters=1)
      (2): Linear(in_features=64, out_features=1, bias=True)
    )
  )


In [ ]:
#hide
def create_test_dataframe_with_identical_rows(nrows = 10):
    repeat = 10
    precursor_df = pd.DataFrame({
        'sequence': ['AGHCEWQMKYR']*nrows,
        'mods': ['Acetyl@Protein N-term;Carbamidomethyl@C;Oxidation@M']*nrows,
        'mod_sites': ['0;4;8']*nrows,
        'nAA': [11]*nrows,
        'rt_norm': [0.6]*nrows
    })
    return precursor_df

repeat_row_df = create_test_dataframe_with_identical_rows()
display(repeat_row_df)

model.train(repeat_row_df, epoch=5, verbose_each_epoch=True)



,sequence,mods,mod_sites,nAA,rt_norm
0,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6
1,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6
2,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6
3,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6
4,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6
5,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6
6,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6
7,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6
8,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6
9,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6


Epoch=5, nAA=11, batch=1, loss=0.5764: 100%|██████████| 1/1 [00:00<00:00, 26.59it/s]


In [ ]:
import numpy as np

def check_that_predicted_rts_are_almost_identical(repeat_row_df):
    repeat_row_df_predict = model.predict(repeat_row_df)
    display(repeat_row_df_predict)
    pred_rts = list(repeat_row_df_predict["rt_pred"])
    first_pred_rt = pred_rts[0]
    control_array = [first_pred_rt for x in range(len(pred_rts))]

    np.testing.assert_almost_equal(pred_rts, control_array)

check_that_predicted_rts_are_almost_identical(repeat_row_df)


,sequence,mods,mod_sites,nAA,rt_norm,rt_pred
0,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6,0.03271
1,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6,0.03271
2,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6,0.03271
3,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6,0.03271
4,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6,0.03271
5,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6,0.03271
6,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6,0.03271
7,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6,0.03271
8,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6,0.03271
9,AGHCEWQMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,0.6,0.03271


In [ ]:
#hide
from peptdeep.pretrained_models import ModelManager
models = ModelManager()
models.load_installed_models()
pep_df = IRT_PEPTIDE_DF.iloc[:7].drop(columns=['irt'])
df_w_rt_prediction = models.rt_model.predict(pep_df)
df_w_irt_prediction_added = models.rt_model.add_irt_column_to_precursor_df(pep_df)

assert df_w_irt_prediction_added.rt_pred.is_monotonic
assert df_w_irt_prediction_added.irt_pred.is_monotonic
df_w_rt_prediction = models.rt_model.predict(IRT_PEPTIDE_DF)
models.rt_model.add_irt_column_to_precursor_df(IRT_PEPTIDE_DF)

,sequence,pep_name,irt,mods,mod_sites,nAA,rt_pred,irt_pred
0,LGGNEQVTR,RT-pep a,-24.92,,,9,0.072804,-28.148849
1,GAGSSEPVTGLDAK,RT-pep b,0.00,,,14,0.271196,2.053492
2,VEATFGVDESNAK,RT-pep c,12.39,,,13,0.332649,11.408902
3,YILAGVENSK,RT-pep d,19.79,,,10,0.400949,21.806524
4,TPVISGGPYEYR,RT-pep e,28.71,,,12,0.438901,27.584271
5,TPVITGAPYEYR,RT-pep f,33.38,,,12,0.489774,35.328937
6,DGLDAASYYAPVR,RT-pep g,42.26,,,13,0.542729,43.390475
7,ADVTPADFSEWSK,RT-pep h,54.62,,,13,0.609782,53.598396
8,GTFIIDPGGVIR,RT-pep i,70.52,,,12,0.757164,76.035216
9,GTFIIDPAAVIR,RT-pep k,87.23,,,12,0.846791,89.679588
